## Feature WOE && IV

In [8]:
import pandas as pd
import numpy as np
import cudf
import seaborn as sns
import matplotlib.pyplot as plt
from tabulate import tabulate


In [2]:
train = pd.read_parquet("../data/train.parquet")


In [3]:
cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
num_cols = [col for col in train.columns if col not in cat_cols and col not in ["customer_ID", "S_2"]]

In [4]:
cont_features = [col for col in train.columns if col in num_cols and len(np.unique(train[col]))<=10]

In [7]:
train = train.fillna("MISSING")

In [5]:
def calcular_woe_iv(dataset, feature, target):
    lst = []
    for i in range(dataset[feature].nunique()):
        val = list(dataset[feature].unique())[i]
        lst.append({
            "Value": val,
            "All": dataset[dataset[feature] == val].count()[feature],
            "Bom": dataset[(dataset[feature] == val) & (dataset[target] == 0)].count()[feature],
            "Mau": dataset[(dataset[feature] == val) & (dataset[target] == 1)].count()[feature]
        })
        
    dset = pd.DataFrame(lst)
    dset["Distr_Bom"] = dset["Bom"] / dset["Bom"].sum()
    dset["Distr_Mau"] = dset["Mau"] / dset["Mau"].sum()
    dset["WoE"] = np.log(dset["Distr_Bom"] / dset["Distr_Mau"])
    dset = dset.replace({"WoE": {np.inf: 0, -np.inf: 0}})
    dset["IV"] = (dset["Distr_Bom"] - dset["Distr_Mau"]) * dset["WoE"]
    iv = dset["IV"].sum()
 #   dset = dset.sort_values(by="WoE")
    
    return dset, iv

In [9]:
df1 = {}
l = 0
for col in cont_features:
    if col == 	"Exited	": continue
    else:
        print("WoE e IV: {}".format(col))
        df, iv = calcular_woe_iv(train, col, "target")
        df1[l] = df
        df1[l]["nome"] = "WOE_"+col
        print(l)
        l = l + 1
        print(tabulate(df, headers="keys"))
        print("IV score: {:.2f}".format(iv))
        print("\n")
        i = 0
        for i in range(0,len(df)):
            train.loc[train[col] ==  df.iloc[i,0], "WOE_"+col] = df.iloc[i,6]

WoE e IV: R_2
0
      Value      All      Bom      Mau    Distr_Bom    Distr_Mau        WoE         IV  nome
--  -------  -------  -------  -------  -----------  -----------  ---------  ---------  -------
 0        0  5296274  4104301  1191973    0.988135      0.865084   0.132993  0.0163648  WOE_R_2
 1        1   235177    49281   185896    0.0118647     0.134916  -2.43108   0.299147   WOE_R_2
IV score: 0.32


WoE e IV: D_51
1
      Value      All      Bom      Mau    Distr_Bom    Distr_Mau        WoE           IV  nome
--  -------  -------  -------  -------  -----------  -----------  ---------  -----------  --------
 0        4    22185    21713      472  0.00522754   0.000342558   2.72525   0.0133128    WOE_D_51
 1        3    90456    88547     1909  0.0213182    0.00138547    2.73352   0.0544866    WOE_D_51
 2        2   322793   311744    11049  0.0750543    0.0080189     2.23641   0.149918     WOE_D_51
 3        1  1290125  1174648   115477  0.282804     0.0838084     1.21622   0

/home/victor/miniconda3/envs/rapidsai/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [11]:
train.to_parquet("../data/train_woe_balanced.parquet")